In [1]:
import basicLib.loadAndTest as orig
import basicLib.featureUpdate as featUp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import *
from pylab import *
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn import metrics
import re
from sklearn.linear_model import LogisticRegression
import basicLib.modelRun as runms
from sklearn.neural_network import MLPClassifier
from sklearn import tree
import time
import clf2net
import clf2netTest
import clf2netTree
from sklearn import ensemble

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
pathTrain = '../input_clean/users_train_norm.csv'
pathTest = '../input_clean/users_test_norm.csv'

users, yRes = orig.loadAndUpdateFeatures(pathTrain, pathTest=pathTest)
orig.initiateUsers(users)
featureList = orig.featureList(usersCol=users.columns)
Xtest = orig.sortFeatures(users,featureList.get())

NameError: name 'featureList' is not defined

In [6]:
pathTrain = '../input_clean/users_train_norm.csv'
pathTest = '../input_clean/users_test_norm.csv'

usersTrain = pd.read_csv(pathTrain)
usersTest = pd.read_csv(pathTest)
yRes = pd.DataFrame(usersTrain['country_destination'])
yFeat = featUp.countriesFeature()
yFeat.update(yRes)
usersTrain = usersTrain.drop('country_destination',axis=1)
users = pd.concat([usersTrain,usersTest],ignore_index=True)
featureStruct=featUp.generateFeatureStruct()
for feature in featureStruct:
    feature.update(users)
orig.initiateUsers(users)
featureList = orig.featureList(usersCol=users.columns)
Xtest = orig.sortFeatures(users,featureList.get())

In [7]:
Xlabel = Xtest
nonCategoryList=[]
for feature, dtype in Xlabel.dtypes.iteritems():
    if dtype.name != 'category':
        nonCategoryList.append(feature)
nonCategoryDF = Xlabel[nonCategoryList]
Xlabel = Xlabel.drop(nonCategoryList, 1)

labels = Xlabel.columns
myLabels = {} # holds the encode labels with value between 0 and n-1
n_values = [] # holds the length of each feature in bits ('gender' with 4 option will be 4 bits)
X = []
for label in Xlabel:
    labelAttr = getattr(Xlabel, label)
    myLabels[label] = preprocessing.LabelEncoder()
    myLabels[label].fit(labelAttr)
    if size(labelAttr.unique()) > 400:
        raise TypeError('The feature:', label, 'is too big to handle as binary')
    n_values.append(size(labelAttr.unique()))
    #transform: turn options into 0 to n-1,  reshape+transpose: align the coulumn so it be easier to add them to the table 
    X.append(myLabels[label].transform(labelAttr).reshape(1,len(labelAttr)).transpose())

#There is no category to change

temp = np.array([]).reshape(len(X[0]),0)
for i in range(len(X)):
    temp = concatenate((temp,X[i]),axis=1)
X = temp

enc = preprocessing.OneHotEncoder(n_values=n_values)
enc.fit(X)
X_byte = enc.transform(X).toarray()

X_byteDF = orig.makeDataFrame(X_byte,labels,n_values)
result = pd.concat([X_byteDF, nonCategoryDF], axis=1)

In [8]:
myDrop = {}
myList=('affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked',
        'first_browser', 'first_device_type', 'gender', 'language',
        'signup_app', 'signup_method', 'signup_flow')
for i in myList:
    attrTest = getattr(usersTest,i)
    attrTrain = getattr(usersTrain,i)
    myDrop[i] = []
    for x in attrTest.unique():
        if x not in attrTrain.unique():
            myDrop[i].append(x)
            print(i,':', x)

first_affiliate_tracked : nan
first_browser : Opera Mobile
first_browser : IBrowse
first_browser : Nintendo Browser
first_browser : CometBird
first_browser : UC Browser
language : -unknown-
signup_method : weibo
signup_flow : 14


In [9]:
for key in myDrop.keys():
    for testCat in myDrop[key]:
        print(key,',', testCat)
        if pd.isnull(testCat): continue
        myInd = myLabels[key].classes_.tolist().index(testCat)
        label = key + '_' + str(myInd)
        result = result.drop(label,axis=1)

first_browser , Opera Mobile
first_browser , IBrowse
first_browser , Nintendo Browser
first_browser , CometBird
first_browser , UC Browser
signup_flow , 14
signup_method , weibo
first_affiliate_tracked , nan
language , -unknown-


In [10]:
orig.normDf(result)
finalTrain = result[0:len(yRes)]
finalTest = result[len(yRes):]

In [13]:
finalTrain.to_csv('../input_clean/users_train_final.csv')
finalTest.to_csv('../input_clean/users_train_final.csv')
yRes.to_csv('../input_clean/yRes_final.csv')